# Text Classification (UAS)

## Data Preparation

download libraries & packages yang dibutuhkan

In [ ]:
# Sentence Transformers: Multilingual Sentence, Paragraph, and Image Embeddings using BERT & Co.
!pip install sentence-transformers
!pip install git+https://github.com/MartinoMensio/spacy-sentence-bert.git

# for model paraphrase-xlm-r-multilingual-v1
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_paraphrase_xlm_r_multilingual_v1-0.1.2.tar.gz # xx_paraphrase_xlm_r_multilingual_v1-0.1.2

# for model distiluse-base-multilingual-cased-v2
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_distiluse_base_multilingual_cased_v2-0.1.2.tar.gz #xx_distiluse_base_multilingual_cased_v2-0.1.2

# Stem for Indonesian Language
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 897.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6c4a1d42cca924d22da35707ef177a1dce6f483a75472ec583e529638ad91537
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Cloning https://github.com/MartinoMensio/spacy-sentence-bert.git to /tmp/pip-req-build-c07ksbqf
  Running command git clone --filter=blob:none --quiet https://github.com/MartinoMensio/spacy-sentence-bert.git /tmp/pip-req-build-c07ksbqf
  Resolved https://github.com/MartinoMensio/spacy-sentence-bert.git to commit 9fb25f92b5be77d42921a66fda2c54caa3199925
  Preparing metadata (setup.py) ... done
  Created wheel for spacy-sentence-bert: filename=spacy_sentence_bert-0.

Impor libraries yang dibutuhkan

In [ ]:
import pandas as pd
import numpy as np
from numpy import array


#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# stemming for Indonesian Language
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('stopwords')


#for model-building
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA
import seaborn as sns  # to plot the heat maps


#for word embedding
import gensim
from gensim.models import Word2Vec

# for NLP use BERT
from sentence_transformers import SentenceTransformer
import spacy_sentence_bert

import warnings
warnings.filterwarnings(action = 'ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Persiapan Dataset

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/tiarh/data/main/detik.csv")
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
df.head(10)

,url,judul,penulis,tanggal,isi,kategori
0,https://health.detik.com/berita-detikhealth/d-...,Viral Minum 4 Butir Obat Sakit Kepala Biar Tid...,Averus Kautsar - detikHealth,"Kamis, 07 Des 2023 16:32 WIB",Jakarta - Viral di media sosial seorang netize...,DetikHealth
1,https://health.detik.com/berita-detikhealth/d-...,"Geger Mycoplasma Pneumonia di Indonesia, Dipas...",Khadijah Nur Azizah - detikHealth,"Kamis, 07 Des 2023 16:00 WIB",Jakarta - Infeksi mycoplasma pneumonia bikin g...,DetikHealth
2,https://health.detik.com/berita-detikhealth/d-...,Sanksi Berat Kemenkes buat RS 'Nakal' Klaim BP...,Vidya Pinandhita - detikHealth,"Kamis, 07 Des 2023 15:30 WIB",Daftar Isi Sanksi Berat buat RS yang 'Nakal' J...,DetikHealth
3,https://health.detik.com/berita-detikhealth/d-...,"Kasus COVID-19 RI Kembali 'Ngegas', WFH Lagi N...",Suci Risanti Rahmadania - detikHealth,"Kamis, 07 Des 2023 15:09 WIB",Daftar Isi Bakal Diterapkan PJJ dan WFH Lagi? ...,DetikHealth
4,https://health.detik.com/true-story/d-7077181/...,Pilu Wanita 24 Tahun Meninggal Akibat Kanker U...,Suci Risanti Rahmadania - detikHealth,"Kamis, 07 Des 2023 14:31 WIB",Daftar Isi Dikenal Sehat dan Bugar Jakarta - S...,DetikHealth
5,https://health.detik.com/berita-detikhealth/d-...,"Mycoplasma Pneumonia Muncul di DKI, Menkes Pas...",Vidya Pinandhita - detikHealth,"Kamis, 07 Des 2023 14:00 WIB",Daftar Isi Gejalanya Relatif Ringan Jakarta - ...,DetikHealth
6,https://health.detik.com/berita-detikhealth/d-...,"Momen Kim Jong Un Menangis, Minta Ibu-ibu di K...",Averus Kautsar - detikHealth,"Kamis, 07 Des 2023 13:30 WIB",Jakarta - Kim Jong-un menangis di depan ribuan...,DetikHealth
7,https://health.detik.com/berita-detikhealth/d-...,Gejala Hari ke Hari 6 Pasien Mycoplasma Pneumo...,Nafilah Sri Sagita K - detikHealth,"Kamis, 07 Des 2023 13:02 WIB",Jakarta - Total ada enam kasus mycoplasma pneu...,DetikHealth
8,https://health.detik.com/berita-detikhealth/d-...,"BPJS Kesehatan Ungkap Temuan Fraud, Ada Klaim ...",Vidya Pinandhita - detikHealth,"Kamis, 07 Des 2023 12:30 WIB",Daftar Isi Kemenkes RI Turun Tangan Jakarta - ...,DetikHealth
9,https://health.detik.com/berita-detikhealth/d-...,"6 Cara Menjaga Kesehatan Paru-paru, Ternyata S...",Atta Kharisma - detikHealth,"Kamis, 07 Des 2023 12:02 WIB",Daftar Isi Cara Menjaga Kesehatan Paru-paru 1....,DetikHealth


In [ ]:
label = df["kategori"]

In [ ]:
print(label.unique)

<bound method Series.unique of 0       DetikHealth
1       DetikHealth
2       DetikHealth
3       DetikHealth
4       DetikHealth
           ...     
2064     DetikSport
2065     DetikSport
2066     DetikSport
2067     DetikSport
2068     DetikSport
Name: kategori, Length: 2069, dtype: object>


In [ ]:
news_datas = df["isi"]

In [ ]:
news_datas.head(10)

0    Jakarta - Viral di media sosial seorang netize...
1    Jakarta - Infeksi mycoplasma pneumonia bikin g...
2    Daftar Isi Sanksi Berat buat RS yang 'Nakal' J...
3    Daftar Isi Bakal Diterapkan PJJ dan WFH Lagi? ...
4    Daftar Isi Dikenal Sehat dan Bugar Jakarta - S...
5    Daftar Isi Gejalanya Relatif Ringan Jakarta - ...
6    Jakarta - Kim Jong-un menangis di depan ribuan...
7    Jakarta - Total ada enam kasus mycoplasma pneu...
8    Daftar Isi Kemenkes RI Turun Tangan Jakarta - ...
9    Daftar Isi Cara Menjaga Kesehatan Paru-paru 1....
Name: isi, dtype: object

## Preprocessing

In [ ]:
def preprocess(text):
    """convert to lowercase, strip and remove punctuations"""
    text = text.lower()

    # remove whitespace
    text = text.strip()
    # compile pattern that have `<.*?>` then replace with ''
    text = re.compile('<.*?>').sub('', text)

    # compile pattern that have punctuation !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~ then replace with ' '
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)

    # replace all one or more space to one space
    text = re.sub('\s+', ' ', text)

    # replace all number 0-9 to one space
    text = re.sub(r"\d+", "", text)

    # replace all first word or space in the beginning of line to ''
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    # replace all digits to one space
    text = re.sub(r'\d',' ',text)

    # replace all one or more space to one space
    text = re.sub(r'\s+',' ',text)

    return text

def stopword(string):
    """Stopword removal like I, The, An, be, is, etc"""
    a = [i for i in string.split() if i not in stopwords.words('indonesian')]
    return ' '.join(a)



Bersihkan kata pada abstrak dengan fungsi ```preprocess()``` untuk menghilangkan angka, tanda baca, dan ubah kata ke huruf kecil

In [ ]:
preprocessed_news = []
for text in news_datas:
    preprocessed_news.append(preprocess(str(text)))

In [ ]:
preprocessed_news = pd.Series(preprocessed_news)

In [ ]:
preprocessed_news[90]

'jakarta manfaat asam folat bagi ibu hamil ramai diperbincangkan di jagat maya saat cawapres nomor urut gibran rakabuming raka salah menyebutkan asam folat menjadi asam sulfat lantas apa sebenarnya manfaat asam folat dan bagaimana cara mendapatkannya perlu diketahui asam folat adalah bentuk sintetik dari vitamin b yang larut dalam air vitamin b juga dikenal sebagai folat atau folasin yang dapat ditemukan pada susu asam folat berperan penting dalam pembentukan sel darah merah mendukung daya tahan tubuh dan menunjang proses tumbuh kembang janin advertisement scroll to continue with content manfaat asam folat bagi ibu hamil adalah mencegah cacat tabung saraf neural tube defect seperti anensefali janin tidak memiliki otak dan spina bifida janin tidak memiliki sumsum tulang belakang selain itu asam folat dapat mencegah anemia defisiensi folat yaitu kondisi ketika tubuh kekurangan sel darah merah yang sehat baca juga salah sebut asam sulfat untuk ibu hamil gibran minta maaf adapun asam folat

Bersihkan kata pada abstrak dengan fungsi ```stopword()``` untuk menghapus kata hubung dan semua kata bantu

In [ ]:
removed_stopword_news = []
for text in preprocessed_news:
    removed_stopword_news.append(stopword(str(text)))

In [ ]:
removed_stopword_news = pd.Series(removed_stopword_news)

In [ ]:
removed_stopword_news[90]

'jakarta manfaat asam folat hamil ramai diperbincangkan jagat maya cawapres nomor urut gibran rakabuming raka salah asam folat asam sulfat lantas manfaat asam folat mendapatkannya asam folat bentuk sintetik vitamin b larut air vitamin b dikenal folat folasin ditemukan susu asam folat berperan pembentukan sel darah merah mendukung daya tahan tubuh menunjang proses tumbuh kembang janin advertisement scroll to continue with content manfaat asam folat hamil mencegah cacat tabung saraf neural tube defect anensefali janin memiliki otak spina bifida janin memiliki sumsum tulang asam folat mencegah anemia defisiensi folat kondisi tubuh kekurangan sel darah merah sehat baca salah asam sulfat hamil gibran maaf asam folat membantu perkembangan otak sumsum tulang janin membantu pembentukan plasenta membantu mencegah preeklamsia kondisi serius kehamilan sumber asam folat diperoleh makanan suplemen makanan mengandung asam folat sayuran hijau bayam kangkung seledri lobak buah buahan jeruk stroberi al

Bersihkan kata dengan fungsi ```to_base_word()``` untuk mengubah kata menjadi kata dasar.

In [ ]:
# sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def to_base_word(string):
    """Change words to base words"""
    string = string.split()
    text = [stemmer.stem(word) for word in string]
    return ' '.join(text)

In [ ]:
removed_stopword_news = []
for text in preprocessed_news:
    removed_stopword_news.append(stopword(str(text)))

cleaned_news = []
for text in removed_stopword_news:
    cleaned_news.append(to_base_word(str(text)))

In [ ]:
cleaned_news = pd.Series(cleaned_news)

In [ ]:
cleaned_news[90]

'jakarta manfaat asam folat hamil ramai bincang jagat maya cawapres nomor urut gibran rakabuming raka salah asam folat asam sulfat lantas manfaat asam folat dapat asam folat bentuk sintetik vitamin b larut air vitamin b kenal folat folasin temu susu asam folat peran bentuk sel darah merah dukung daya tahan tubuh tunjang proses tumbuh kembang janin advertisement scroll to continue with content manfaat asam folat hamil cegah cacat tabung saraf neural tube defect anensefali janin milik otak spina bifida janin milik sumsum tulang asam folat cegah anemia defisiensi folat kondisi tubuh kurang sel darah merah sehat baca salah asam sulfat hamil gibran maaf asam folat bantu kembang otak sumsum tulang janin bantu bentuk plasenta bantu cegah preeklamsia kondisi serius hamil sumber asam folat oleh makan suplemen makan kandung asam folat sayur hijau bayam kangkung seledri lobak buah buah jeruk stroberi alpukat kacang kacang kacang kedelai kacang hijau kacang polong kembang kol brokoli telur susu ba

## Evaluation

### Evaluation Preparation

Label ```y```

In [ ]:
print(label.unique())

['DetikHealth' 'DetikFood' 'DetikSport']


Data ```X```

In [ ]:
cleaned_news

0       jakarta viral media sosial netizen media sosia...
1       jakarta infeksi mycoplasma pneumonia bikin geg...
2       daftar isi sanksi berat rs nakal jakarta bpjs ...
3       daftar isi terap pjj wfh jakarta covid indones...
4       daftar isi kenal sehat bugar jakarta wanita na...
                              ...                        
2064    hangzhou kontingen indonesia capai target meda...
2065    jakarta tim borneo basketball usung target jua...
2066    jakarta november tim wajib usul nama nama main...
2067    cesson sevigne anthony sinisuka ginting lang e...
2068    jakarta medical director jakarta marathon dr a...
Length: 2069, dtype: object

Buat *dataframe* baru dari data yang sudah dibersihkan

In [ ]:
new_data = {"isi" : cleaned_news, "kategori" : label}
new_df = pd.concat(new_data, axis = 1)

In [ ]:
new_df

,isi,kategori
0,jakarta viral media sosial netizen media sosia...,DetikHealth
1,jakarta infeksi mycoplasma pneumonia bikin geg...,DetikHealth
2,daftar isi sanksi berat rs nakal jakarta bpjs ...,DetikHealth
3,daftar isi terap pjj wfh jakarta covid indones...,DetikHealth
4,daftar isi kenal sehat bugar jakarta wanita na...,DetikHealth
...,...,...
2064,hangzhou kontingen indonesia capai target meda...,DetikSport
2065,jakarta tim borneo basketball usung target jua...,DetikSport
2066,jakarta november tim wajib usul nama nama main...,DetikSport
2067,cesson sevigne anthony sinisuka ginting lang e...,DetikSport


In [ ]:
new_df.to_csv("detik_clean.csv", index=False)

Load model dari https://github.com/MartinoMensio/spacy-sentence-bert/ untuk melakukan pelatihan setiap kata dan acuan untuk merubah kata menjadi vektor

In [ ]:
# for model distiluse-base-multilingual-cased-v2
nlp = spacy_sentence_bert.load_model('xx_distiluse_base_multilingual_cased_v2')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

Buat kolom ```df['vector']``` dimana setiap berita akan direpresentasikan dalam bentuk vektor

In [ ]:
new_df['vector'] = new_df['isi'].apply(lambda x: nlp(x).vector)

In [ ]:
new_df

,isi,kategori,vector
0,jakarta viral media sosial netizen media sosia...,DetikHealth,"[0.021346692, 0.015949566, -0.024077056, -0.04..."
1,jakarta infeksi mycoplasma pneumonia bikin geg...,DetikHealth,"[0.03337458, -0.006035795, -0.006131712, -0.02..."
2,daftar isi sanksi berat rs nakal jakarta bpjs ...,DetikHealth,"[0.0033310342, 0.07816858, 0.022679642, -0.029..."
3,daftar isi terap pjj wfh jakarta covid indones...,DetikHealth,"[0.01652688, 0.05699369, 0.018889554, -0.01414..."
4,daftar isi kenal sehat bugar jakarta wanita na...,DetikHealth,"[0.03013078, 0.059756313, 0.02502539, 0.004663..."
...,...,...,...
2064,hangzhou kontingen indonesia capai target meda...,DetikSport,"[0.016679322, 0.05699505, -0.03871123, -0.0067..."
2065,jakarta tim borneo basketball usung target jua...,DetikSport,"[0.019712722, 0.055120107, 0.0047338894, -0.02..."
2066,jakarta november tim wajib usul nama nama main...,DetikSport,"[0.065902494, 0.07774033, 0.00633163, -0.03983..."
2067,cesson sevigne anthony sinisuka ginting lang e...,DetikSport,"[0.01959582, 0.033822246, -0.007032108, -0.017..."


Pisah data X & label y


*   ```X_train``` train X data
*   ```X_test``` test X data


*   ```y_train``` train y data
*   ```y_test``` test y data



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df['vector'].tolist(), new_df['kategori'].tolist(), test_size=0.33, random_state=42)

### Train with machine learning classifier.

#### Support Vector Classifier(SVM)

#### Gaussian Naive Bayes

In [ ]:
modelGNB = GaussianNB()

modelGNB.fit(X_train, y_train)
y_pred = modelGNB.predict(X_test)
accuracyGNB = accuracy_score(y_pred, y_test)
print("Accuracy Score:", accuracyGNB)

Accuracy Score: 0.9546120058565154


#### K-Nearest Neighbors


```n_neighbors = 3```

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)

Accuracy Score: 0.9619326500732065


### Classification Test

Uji coba berita baru terhadap model yang sudah dilatih


*   ```news``` untuk data
*   ```labels``` untuk kategori



In [ ]:
news = """Sejumlah bayi prematur yang dievakuasi dari Gaza pada November lalu dirawat di RS New Administrative Capital (NAC) di Kairo, Mesir. Begini potretnya.

Baca artikel detikHealth, "Melihat Perawatan Bayi Prematur yang Dievakuasi dari Gaza" selengkapnya https://health.detik.com/fotohealth/d-7077578/melihat-perawatan-bayi-prematur-yang-dievakuasi-dari-gaza.

Download Apps Detikcom Sekarang https://apps.detik.com/detik/"""

print(f"Predicted Label: {knn.predict(nlp(news).vector.reshape(1, -1))[0]} \n")

Predicted Label: DetikHealth 

